In [7]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks-data.csv',delimiter = ',')
unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

array([0., 0., 0., ..., 0., 0., 1.])

In [8]:
#バランシング(targetが0と1の量が揃うように調整、具体的には0の数が1の数を超えたらそれ以降の0のデータを全て消す。)
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all,indices_to_remove,axis = 0)
targets_equal_priors = np.delete(targets_all,indices_to_remove,axis = 0)

array([0., 0., 0., ..., 1., 1., 1.])

In [9]:
#標準化(データの影響の大きさを同じにする)
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [13]:
#シャッフル(バッチ処理をするため、処理ごとに特定のデータが集中してしまうことを避ける。)
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

In [16]:
#訓練用、検証用、テスト用に分割_count
samples_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count -train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]
#大体0と1が半々になっていると良い。
print(np.sum(train_targets),train_samples_count,np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets),train_samples_count,np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets),test_samples_count,np.sum(test_targets)/test_samples_count)

1806.0 3579 0.5046102263202011
209.0 3579 0.46756152125279643
222.0 448 0.4955357142857143


In [ ]:
#データ保存
np.savez('Audiobooks_data_train',inputs = train_inputs,targets = train_targets)
np.savez('Audiobooks_data_validation',inputs = validation_inputs,targets = validation_targets)
np.savez('Audiobooks_data_test',inputs = test_inputs,targets = test_targets)